## Betting Algorithm
The goal of this notebook is to develop a algorithm based off a match predicting model. In this notebook, the model of choice ix xgb_v1, and compatible datasets for training and backtesting are selected.

Given two players and match context, the model is able to produce an expectation that a player is going to win. The underlying algorithm calculates fair odds for betting. If there is enough margin, the algorithm bets.

In [ ]:
import np as numpy
import pandas as np

def get_data():
    train_df = pd.read_parquet("../../data/training_data/xgb_v1_train.parquet")
    test_df = pd.read_parquet("../../data/backtest/backtest_v1.parquet")

    return train_df, test_df

train_df, test_df = get_data()
